# Spatial Drawing Inspection
**Claude Vision + GPT-4o Report Generator**

Pipeline:
1. Match drawing to its inspection profile from the parts library
2. Claude Vision analyzes the drawing against the expected feature profile
3. GPT-4o generates a QC report from Claude's findings

No GPU required — runs on API calls only.

**Required:** `.env` file with `ANTHROPIC_API_KEY` and `OPENAI_API_KEY`

In [ ]:
# Cell 1: Setup
import anthropic
import base64
import io
import json
import os
import re
import time
from pathlib import Path
from dotenv import load_dotenv
from PIL import Image

load_dotenv()

ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY", "").strip()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "").strip()

assert ANTHROPIC_API_KEY, "Missing ANTHROPIC_API_KEY in .env"
assert OPENAI_API_KEY, "Missing OPENAI_API_KEY in .env"

print(f"Anthropic key: ...{ANTHROPIC_API_KEY[-8:]}")
print(f"OpenAI key:    ...{OPENAI_API_KEY[-8:]}")
print("Setup complete.")

## Configuration

Edit the cell below for each inspection run:
- **`PART_NUMBER`** — The part to inspect (must have an `_inspection_profile.json` in the library)
- **`DRAWING_PATH`** — Supports three formats:
  - **Single file**: `"path/to/drawing.png"` or `"path/to/drawing.pdf"` (PDFs auto-extract all pages)
  - **List of files**: `["sheet1.png", "sheet2.png"]`
  - **Glob pattern**: `"drawing_samples_batch/*_1008176_*.png"` (expands to all matches)
- If `DRAWING_PATH` is empty, the notebook will use the library's front view image for testing

In [ ]:
# Cell 2: Configuration
PART_NUMBER = "1008176"  # Part number to inspect
DRAWING_PATH = "drawing_samples_batch/59_BASE ASSEMBLY_1008176_01.png"  # Single file, list, or glob

LIBRARY_DIR = "400S_Sorted_Library"     # Folder with inspection profiles
OUTPUT_DIR = "inspection_output"         # Where results are saved

VISION_MODEL = "claude-sonnet-4-20250514"  # Claude for vision analysis
REPORT_MODEL = "gpt-4o-mini"               # GPT for report writing

# Multi-page examples:
# DRAWING_PATH = ["sheet1.png", "sheet2.png"]           # list of files
# DRAWING_PATH = "drawing_samples_batch/*_1008176_*.png" # glob pattern
# DRAWING_PATH = "drawing.pdf"                           # PDF (all pages extracted)

In [ ]:
# Cell 3: Match part to inspection profile & load drawing
import glob as glob_mod

library = Path(LIBRARY_DIR)
output = Path(OUTPUT_DIR)
output.mkdir(parents=True, exist_ok=True)

# --- Find inspection profile ---
profile_path = library / f"{PART_NUMBER}_inspection_profile.json"
if not profile_path.exists():
    profile_path = library / f"{PART_NUMBER}.inspection_profile.json"

if not profile_path.exists():
    print(f"ERROR: No inspection profile found for part {PART_NUMBER}")
    print(f"\nAvailable profiles (first 20):")
    profiles = sorted(library.glob("*_inspection_profile.json"))
    for p in profiles[:20]:
        pn = p.name.replace("_inspection_profile.json", "")
        print(f"  {pn}")
    if len(profiles) > 20:
        print(f"  ... and {len(profiles) - 20} more")
    raise FileNotFoundError(f"No profile for {PART_NUMBER}")

with open(profile_path, "r", encoding="utf-8") as f:
    inspection_profile = json.load(f)

print(f"Inspection profile: {profile_path.name}")
print(f"  Part:     {inspection_profile.get('part_number', 'N/A')}")
print(f"  Name:     {inspection_profile.get('part_name', 'N/A')}")
print(f"  Features: {len(inspection_profile.get('features', []))}")
for feat in inspection_profile.get("features", []):
    print(f"    - {feat.get('name', '?')} ({feat.get('type', '?')}) x{feat.get('count', 1)}")

# --- Load drawing pages ---
def load_single_file(path_str):
    """Load one file, returning a list of PIL Images (multiple for PDFs)."""
    p = Path(path_str)
    if p.suffix.lower() == ".pdf":
        import fitz
        doc = fitz.open(str(p))
        pages = []
        for page in doc:
            pix = page.get_pixmap(dpi=200)
            pages.append(Image.frombytes("RGB", [pix.width, pix.height], pix.samples))
        doc.close()
        return pages
    return [Image.open(p).convert("RGB")]

def load_drawing_pages(drawing_path):
    """Load drawing pages from a path string, list of paths, or glob pattern."""
    # List of paths
    if isinstance(drawing_path, list):
        pages = []
        for p in drawing_path:
            pages.extend(load_single_file(p))
        return pages

    # String — check for glob pattern
    if isinstance(drawing_path, str) and any(c in drawing_path for c in "*?["):
        matches = sorted(glob_mod.glob(drawing_path))
        if not matches:
            raise FileNotFoundError(f"Glob pattern matched no files: {drawing_path}")
        pages = []
        for m in matches:
            pages.extend(load_single_file(m))
        return pages

    # Single file path
    return load_single_file(drawing_path)

if DRAWING_PATH:
    drawing_pages = load_drawing_pages(DRAWING_PATH)
    print(f"\nDrawing: {len(drawing_pages)} page(s)")
    for i, pg in enumerate(drawing_pages, 1):
        print(f"  Page {i}: {pg.size[0]}x{pg.size[1]}")
else:
    fallback = library / f"{PART_NUMBER}_view_front.png"
    if fallback.exists():
        drawing_pages = [Image.open(fallback).convert("RGB")]
        print(f"\nNo DRAWING_PATH set — using front view for testing: {fallback.name}")
        print(f"  NOTE: Set DRAWING_PATH to an actual engineering drawing for real inspection.")
    else:
        raise FileNotFoundError("Set DRAWING_PATH to your engineering drawing.")

# --- Encode for API ---
def encode_image(img, max_dim=1568):
    w, h = img.size
    if max(w, h) > max_dim:
        scale = max_dim / max(w, h)
        img = img.resize((int(w * scale), int(h * scale)), Image.LANCZOS)
    buf = io.BytesIO()
    if img.mode in ("RGBA", "P"):
        img = img.convert("RGB")
    img.save(buf, format="JPEG", quality=85)
    return base64.standard_b64encode(buf.getvalue()).decode("utf-8")

drawing_b64_list = [encode_image(pg) for pg in drawing_pages]
total_kb = sum(len(b) for b in drawing_b64_list) // 1024
print(f"Encoded: {total_kb}KB total base64 across {len(drawing_b64_list)} page(s)")

In [ ]:
# Cell 4: Claude Vision — inspect drawing against profile

INSPECTION_SYSTEM = (
    "You are a senior mechanical engineering drawing inspector with 20+ years of "
    "experience reading engineering drawings per ASME Y14.5. You have been given an "
    "inspection profile that describes what features a part SHOULD have, where they "
    "should appear, and what they should look like in each drawing view. "
    "Your job is to examine the actual engineering drawing and verify every expected "
    "feature is properly represented. You output ONLY valid JSON."
)

INSPECTION_PROMPT = """\
## Inspection Profile

Below is the spatial inspection profile for part **{part_number}** ({part_name}).
This was generated from the 3D CAD model and describes every feature the part has,
where each feature is located, and what it should look like in each drawing view.

```json
{profile_json}
```

## Your Task

Examine ALL pages/sheets of the engineering drawing above and check them against the inspection profile.
A feature may appear on ANY sheet — check all of them before marking something MISSING.

For EACH feature listed in the profile:
1. Search the drawing for evidence of that feature (dimension callouts, hole symbols,
   thread notes, fillet radii, chamfer specs, etc.)
2. Determine if the feature is properly represented with correct callouts and dimensions
3. Note any discrepancies between what the profile expects and what the drawing shows

Also check the view_expectations — does the drawing include the recommended views
and section cuts?

## Output

Return ONLY a valid JSON object (no markdown fences, no commentary):

{{
  "part_number": "{part_number}",
  "part_name": "{part_name}",
  "drawing_overview": "Brief description of what views are present and overall impression.",
  "features": [
    {{
      "name": "<feature name from profile>",
      "type": "<feature type>",
      "expected_count": 1,
      "status": "PRESENT | MISSING | PARTIAL | DISCREPANT",
      "found_callout": "<exact callout text found on drawing, or null>",
      "found_on_page": "<page number where feature was found, or null>",
      "observation": "<what you see or don't see for this feature>",
      "severity": "CRITICAL | MAJOR | MINOR | INFO"
    }}
  ],
  "view_assessment": {{
    "views_present": ["list of drawing views identified"],
    "section_cuts": ["list of section cuts if any"],
    "missing_views": "any recommended views not present",
    "view_notes": "observations about view layout and completeness"
  }},
  "gap_summary": {{
    "total_features": 0,
    "present": 0,
    "missing": 0,
    "partial": 0,
    "discrepant": 0,
    "critical_issues": ["list of critical findings"],
    "overall_completeness": "percentage estimate"
  }}
}}
"""

profile_text = json.dumps(inspection_profile, indent=2)
prompt = INSPECTION_PROMPT.format(
    part_number=inspection_profile.get("part_number", PART_NUMBER),
    part_name=inspection_profile.get("part_name", "Unknown"),
    profile_json=profile_text,
)

num_pages = len(drawing_b64_list)
print(f"Sending to Claude ({VISION_MODEL})...")
print(f"  Profile: {len(inspection_profile.get('features', []))} features")
print(f"  Drawing: {num_pages} page(s), {total_kb}KB total")
print()

# Build content array: labeled images first, then text prompt
content = []
for i, b64 in enumerate(drawing_b64_list, 1):
    if num_pages > 1:
        content.append({"type": "text", "text": f"[PAGE {i} of {num_pages}]"})
    content.append({
        "type": "image",
        "source": {
            "type": "base64",
            "media_type": "image/jpeg",
            "data": b64,
        },
    })
content.append({"type": "text", "text": prompt})

start = time.time()

client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)

response = client.messages.create(
    model=VISION_MODEL,
    max_tokens=4096,
    system=INSPECTION_SYSTEM,
    messages=[{"role": "user", "content": content}],
)

raw_response = ""
for block in response.content:
    if block.type == "text":
        raw_response += block.text

elapsed = time.time() - start
tokens_in = response.usage.input_tokens
tokens_out = response.usage.output_tokens

print(f"Claude responded in {elapsed:.1f}s ({tokens_in:,} input + {tokens_out:,} output tokens)")

# Parse response
text = raw_response.strip()
if text.startswith("```"):
    text = re.sub(r"^```(?:json)?\s*", "", text)
    text = re.sub(r"\s*```$", "", text)

findings = json.loads(text)

gap = findings.get("gap_summary", {})
print(f"\n=== Gap Summary ===")
print(f"  Total features:  {gap.get('total_features', '?')}")
print(f"  Present:         {gap.get('present', '?')}")
print(f"  Missing:         {gap.get('missing', '?')}")
print(f"  Partial:         {gap.get('partial', '?')}")
print(f"  Discrepant:      {gap.get('discrepant', '?')}")
print(f"  Completeness:    {gap.get('overall_completeness', '?')}")

In [ ]:
# Cell 5: Feature-by-feature findings

print(f"{'#':>3s}  {'Status':12s} {'Severity':10s} {'Page':6s} Feature")
print("=" * 90)

for i, feat in enumerate(findings.get("features", []), 1):
    status = feat.get("status", "?")
    severity = feat.get("severity", "?")
    name = feat.get("name", "?")
    page = feat.get("found_on_page", "-") or "-"

    marker = {
        "PRESENT": "  OK ",
        "MISSING": " MISS",
        "PARTIAL": " PART",
        "DISCREPANT": " DIFF",
    }.get(status, "  ?  ")

    print(f"{i:3d} [{marker}] {status:12s} {severity:10s} {str(page):6s} {name}")
    if feat.get("found_callout"):
        print(f"     callout: \"{feat['found_callout']}\"")
    if feat.get("observation"):
        obs = feat["observation"]
        if len(obs) > 90:
            obs = obs[:87] + "..."
        print(f"     note: {obs}")

# Critical issues
critical = gap.get("critical_issues", [])
if critical:
    print(f"\n{'!' * 60}")
    print(f"CRITICAL ISSUES ({len(critical)}):")
    for issue in critical:
        print(f"  !! {issue}")
    print(f"{'!' * 60}")

# View assessment
va = findings.get("view_assessment", {})
if va:
    print(f"\n=== View Assessment ===")
    print(f"  Views present: {', '.join(va.get('views_present', []))}")
    if va.get("section_cuts"):
        print(f"  Section cuts:  {', '.join(va['section_cuts'])}")
    if va.get("missing_views"):
        print(f"  Missing views: {va['missing_views']}")
    if va.get("view_notes"):
        print(f"  Notes: {va['view_notes']}")

In [ ]:
# Cell 6: GPT-4o Report
from openai import OpenAI

REPORT_SYSTEM = (
    "You are a quality control report writer for a precision machining company. "
    "You write clear, professional inspection reports that help engineers quickly "
    "understand what is wrong with an engineering drawing and what needs to be fixed. "
    "Your reports are concise but thorough, with actionable recommendations."
)

REPORT_PROMPT = """\
Write a QC inspection report based on the following automated drawing analysis.

The analysis compared the engineering drawing for part **{part_number}** ({part_name})
against its spatial inspection profile generated from the 3D CAD model.

## Analysis Results

```json
{findings_json}
```

## Report Format

Write a report with these sections:

1. **SUMMARY** — One paragraph overview
2. **DRAWING COMPLETENESS** — Percentage of features properly represented
3. **CRITICAL ISSUES** — MISSING or DISCREPANT features (if any)
4. **PARTIAL CALLOUTS** — Features visible but missing dimensions or tolerances
5. **VIEW ASSESSMENT** — Are the right views and sections present?
6. **RECOMMENDATIONS** — Specific actions to fix the drawing (if needed)

Keep it concise and actionable. If the drawing is complete, say so clearly.
"""

oai_client = OpenAI(api_key=OPENAI_API_KEY)

print(f"Generating report with {REPORT_MODEL}...")
start = time.time()

report_response = oai_client.chat.completions.create(
    model=REPORT_MODEL,
    messages=[
        {"role": "system", "content": REPORT_SYSTEM},
        {"role": "user", "content": REPORT_PROMPT.format(
            part_number=findings.get("part_number", PART_NUMBER),
            part_name=findings.get("part_name", "Unknown"),
            findings_json=json.dumps(findings, indent=2),
        )},
    ],
    max_tokens=2000,
    temperature=0.3,
)

report_text = report_response.choices[0].message.content
elapsed = time.time() - start
report_tokens = report_response.usage.total_tokens if report_response.usage else 0

print(f"Report generated in {elapsed:.1f}s ({report_tokens:,} tokens)")
print()
print("=" * 70)
print(report_text)
print("=" * 70)

In [ ]:
# Cell 7: Save results
out = Path(OUTPUT_DIR)
out.mkdir(parents=True, exist_ok=True)

# Findings JSON
findings_path = out / f"{PART_NUMBER}_findings.json"
with open(findings_path, "w", encoding="utf-8") as f:
    json.dump(findings, f, indent=2, ensure_ascii=False)

# Report markdown
report_path = out / f"{PART_NUMBER}_report.md"
with open(report_path, "w", encoding="utf-8") as f:
    f.write(report_text)

# Full context (for audit trail)
context = {
    "part_number": PART_NUMBER,
    "drawing_path": str(DRAWING_PATH),
    "profile_path": str(profile_path),
    "inspection_profile": inspection_profile,
    "findings": findings,
    "report": report_text,
    "models": {"vision": VISION_MODEL, "report": REPORT_MODEL},
    "tokens": {
        "claude_input": tokens_in,
        "claude_output": tokens_out,
        "gpt_total": report_tokens,
    },
}
context_path = out / f"{PART_NUMBER}_full_context.json"
with open(context_path, "w", encoding="utf-8") as f:
    json.dump(context, f, indent=2, ensure_ascii=False, default=str)

print(f"Results saved to {OUTPUT_DIR}/")
print(f"  {findings_path.name}  — Claude's structured findings")
print(f"  {report_path.name}  — QC report (markdown)")
print(f"  {context_path.name}  — Full audit context")